In [ ]:
import pandas as pd
import os
import numpy as np

pd.set_option('display.max_columns', None)

# --- 1. CARREGAMENTO DAS TRÊS FONTES DE DADOS ---
caminho_ofertas = '../../../planilhas/limpo/modulo_3/limpar_e_padronizar_colunas/ofertas_limpo.csv'
caminho_inscricoes_geral = '../../../planilhas/limpo/modulo_3/limpar_e_padronizar_colunas/inscritos_limpo.csv'
caminho_candidatos_validado= "../../../planilhas/processado/modulo_3/funil_e_candidatos_unicos/candidatos_unicos_processado.csv"

dfo = pd.read_csv(caminho_ofertas)
dfi_geral = pd.read_csv(caminho_inscricoes_geral)
dfi_candidatos_unicos= pd.read_csv(caminho_candidatos_validado)

'''
APENAS nome das 10 areas gerais CINE
'''


coluna_regiao = 'regiao'

substituicao_nome_colunas_df_limpo= {

'regiao_residencia':coluna_regiao,
'regiao_ies':coluna_regiao,

}


for df_temp in [dfo,dfi_geral,dfi_candidatos_unicos]:

    df_temp.rename(columns=substituicao_nome_colunas_df_limpo,inplace=True)



#df_temp aponta pro msm objeto q dfo, e df_temp = df_temp.rename... eh uma copia da memoria do df inteiro
# em outro local da memoria, msm q mudou apenas o nome da coluan ,e reatribui a variavel df_temp
# porem agra dfo aponta pro antigo e df_temp pro novo...
# se quiser pode tentar usar inplace=true q modifica o objeto original, pois msm q fosse
# dfo = dfo.rename tb faria um novo df na memoria ram...

# --- 2. CÁLCULO DE CADA ETAPA DO FUNIL ---
chave_agrupamento = ['ano', 'semestre', 'nome_cine_area_geral',coluna_regiao]
ordem_sort = ['ano', 'semestre', 'id_estudante', 'situacao_fies']
subset_drop = ['ano', 'semestre', 'id_estudante']





# --- Etapas de Ofertas ---
df_vagas_ofertadas = dfo.groupby(chave_agrupamento, as_index=False)['vagas_fies'].sum()

df_vagas_ocupadas = dfo.groupby(chave_agrupamento, as_index=False)['vagas_ocupadas'].sum()



# --- Etapas de Inscrições (Gerais) ---
df_inscritos_geral = (dfi_geral
                      .groupby(chave_agrupamento, as_index=False)
                      .size()
                      .rename(columns={'size': 'Inscritos_Geral'}))





# --- Etapas de Candidatos Únicos (Gerais)


df_candidatos_unicos_geral = (dfi_candidatos_unicos
                              .groupby(chave_agrupamento, as_index=False)
                              .size()
                              .rename(columns={'size': 'Candidatos_Unicos_Geral'}))



# --- Etapas "CONCORRENDO por nota enem > nota corte grupo" de múltiplas fontes ---

# garantir que sao numeros media do enem e a nota de corte e padrao brasileiro
for df_temp in [dfi_geral, dfi_candidatos_unicos]:

    df_temp['media_enem'] = (pd
                                  .to_numeric(df_temp['media_enem']
                                              .astype(str)
                                              .str
                                              .replace(',', '.'), errors='coerce'))
    

    df_temp['nota_corte_gp'] = (pd
                                               .to_numeric(df_temp['nota_corte_gp']
                                                           .astype(str)
                                                           .str
                                                           .replace(',', '.'), errors='coerce'))




# --- Etapa: candidatos unicos com nt suficiente (Candidatos Únicos)-

FILTRO_TAL= dfi_candidatos_unicos['media_enem'] >= dfi_candidatos_unicos['nota_corte_gp']

df_base_candidatos_unicos_nota_suficiente = dfi_candidatos_unicos[FILTRO_TAL].copy()



df_candidatos_unicos_nota_suficiente = (df_base_candidatos_unicos_nota_suficiente
                        .groupby(chave_agrupamento, as_index=False)
                        .size()
                        .rename(columns={'size': 'candidatos_unicos_com_nota_suficiente'}))






# --- Etapa: Inscritos  com Nota Suficiente ---

FILTRO_TAL= dfi_geral['media_enem'] >= dfi_geral['nota_corte_gp']

df_inscritos_com_nota_base = dfi_geral[FILTRO_TAL].copy()

df_inscritos_com_nota = (df_inscritos_com_nota_base
                         .groupby(chave_agrupamento, as_index=False)
                         .size()
                         .rename(columns={'size': 'inscritos_com_nota_suficiente'}))




# --- 3. JUNÇÃO FINAL ---

df_final = df_vagas_ofertadas


dfs_para_juntar = [

#df_vagas_ofertadas, pq df_final o primeiro ja eh df vagas ofertas
df_inscritos_geral,
df_inscritos_com_nota,
df_candidatos_unicos_geral,
df_candidatos_unicos_nota_suficiente,
df_vagas_ocupadas,

]


for df_etapa in dfs_para_juntar:
    df_final = pd.merge(df_final, df_etapa, on=chave_agrupamento, how='outer')

df_final = df_final.fillna(0)

df_final

<ipython-input-1-9d179704e41f>:13: DtypeWarning: Columns (31,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  dfo = pd.read_csv(caminho_ofertas)
<ipython-input-1-9d179704e41f>:14: DtypeWarning: Columns (25,37,38,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  dfi_geral = pd.read_csv(caminho_inscricoes_geral)
<ipython-input-1-9d179704e41f>:15: DtypeWarning: Columns (37,38,49) have mixed types. Specify dtype option on import or set low_memory=False.
  dfi_candidatos_unicos= pd.read_csv(caminho_candidatos_validado)


,ano,semestre,nome_cine_area_geral,regiao,vagas_fies,Inscritos_Geral,inscritos_com_nota_suficiente,Candidatos_Unicos_Geral,candidatos_unicos_com_nota_suficiente,vagas_ocupadas
0,2019.0,1.0,"Agricultura, silvicultura, pesca e veterinária",Centro-Oeste,1973.0,2549,490,1611,364,214.0
1,2019.0,1.0,"Agricultura, silvicultura, pesca e veterinária",Nordeste,3559.0,4492,1761,2701,1095,660.0
2,2019.0,1.0,"Agricultura, silvicultura, pesca e veterinária",Norte,951.0,2260,601,1306,350,225.0
3,2019.0,1.0,"Agricultura, silvicultura, pesca e veterinária",Sudeste,5398.0,11080,1837,6418,1257,696.0
4,2019.0,1.0,"Agricultura, silvicultura, pesca e veterinária",Sul,2697.0,3301,598,2186,440,332.0
...,...,...,...,...,...,...,...,...,...,...
295,2021.0,2.0,Serviços,Centro-Oeste,1919.0,165,144,98,88,17.0
296,2021.0,2.0,Serviços,Nordeste,5993.0,716,632,373,332,80.0
297,2021.0,2.0,Serviços,Norte,1890.0,297,268,147,138,30.0
298,2021.0,2.0,Serviços,Sudeste,8721.0,688,643,353,329,78.0


In [ ]:
df_final.to_csv('../../../planilhas/processado/modulo_3/funil_e_candidatos_unicos/funil_por_regiao.csv', index=False)